#### Day 2 - B
Determine how many rows are "safe" using the following criteria:
* All numbers in the row are either all ascending or descending
* Each number is between 1-3 higher/lower than the previous
* One error is allowed per line

In [26]:
#Import Libraries and settings

settings = {
    "day": 2,
    "test_data": 0
}

In [27]:
#Load Input
def load_input(settings):
    #Derrive input file name
    if settings["test_data"]:
        data_subdir = "test"
    else:
        data_subdir = "actual"

    data_fp = f"./../input/{data_subdir}/{settings["day"]}.txt"

    #Open and read the file
    with open(data_fp) as f:
        lines = f.read().split('\n')

    #Split each line by numbers and convert to numeric values
    lines = [x.split(" ") for x in lines]
    lines = [[int(y) for y in line] for line in lines]

    return lines

data_in = load_input(settings)

In [28]:
print(data_in[0:5])

[[44, 47, 50, 51, 53, 54, 53], [70, 73, 75, 77, 80, 81, 84, 84], [1, 3, 4, 7, 10, 13, 16, 20], [47, 49, 52, 53, 55, 57, 60, 65], [69, 70, 71, 70, 71]]


In [31]:
#Compare two numbers to check which is bigger
def compare_nums(x, y):
    if x < y:
        return 1
    elif x > y:
        return -1
    #If x = y then return 0
    else:
        return 0

#Determine the direction of the list by sampling 4 items in the list
def determine_dir(nums):
    #Evaluate each subset of 3 seperately
    votes = []
    for i in range(0, 4):
        #Create sublist excluding element i
        sublist = nums[0:i] + nums[i+1:4]

        #Process sublist for direction
        #Step 1 is the first item compared to the second
        step_1 = compare_nums(sublist[0], sublist[1])
        #Step 2 is the second item compared to the third
        step_2 = compare_nums(sublist[1], sublist[2])

        #If step 1 and step 2 agree then the vote is 2 or -2
        #If they disagree then the vote is 0
        #If step 1 has a direction and step 2 does not, use step 1 as the vote
        #If neither step 1 or step 2 has a direction then the vote is 0
        votes.append(step_1+step_2)

    #Sum the votes from the subsets to see what the dominant direction is
    res = sum(votes)

    #Normalise result to 1, -1, or 0
    return compare_nums(0, res)

#Evaluate if the difference in magnitude between 2 numbers is allowed
def check_difference(x, y, dir, low=1, high=3):
    diff = (y - x)*dir
    if diff < low or diff > high:
        return False
    else:
        return True

#Process a line to determine if a line is safe
def process_line(line, allow_errs=False, dir=None):

    #Determine the direction for the line
    if dir is None:
        dir = determine_dir(line[0:4])

    #If no conclusive direction, row cannot be "safe"
    if dir == 0:
        return False

    #Iterate through nums
    for idx, num in enumerate(line):
        #First number can't fail
        if idx == 0:
            prev = num
        else:
            #Check the current number is between 1 and 3 higher/lower than the previous
            if not check_difference(prev, num, dir):
                #If enabled, remove potential errorous figures and check the remaining list
                if allow_errs:
                    #If this is the final element of the list then assumed true
                    if idx == len(line) - 1:
                        return True

                    #Check the remainder of the list with the dampening adjustment
                    #Do this but considering the remainder of the list which each potential errorous number missing
                    #If the previous number is being removed (sublist_2) 
                    # then the current num needs to be compared with the second previous number (prefix_2)
                    # If the error is between the first and second number in the list (idx == 1), 
                    # the second number becomes the first in the list and does not need to be compared with earlier elements
                    if idx == 1:
                        prefix_2 = [num]
                    else:
                        prefix_2 = [line[idx-2], num]
                    
                    sublist_1 = [prev] + line[idx+1:]
                    sublist_2 = prefix_2 + line[idx+1:]

                    #Process the remainder of the potential lists and check if either gives a valid solution
                    #Since an infraction was already found, allow_errs is set to False here
                    if (
                        process_line(sublist_1, dir=dir, allow_errs=False) 
                        or process_line(sublist_2, dir=dir, allow_errs=False)
                        ):
                        return True
                    
                return False
          
            #Set prev for the next loop iteration
            prev = num

    #If the code processes the entire loop, the line is "safe"
    return True

def process_input(lines):
    #Process each line and count the safe rows
    count = 0

    for idx, line in enumerate(lines):
        res = process_line(line, allow_errs=True)
        count += int(res)

    return count

In [32]:
print(process_input(data_in))

621
